<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [2]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.0
Pandas:  1.0.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [0]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [6]:
#ANSWER:
bikes = pd.read_table('../../data/bikeshare.csv', header = 0,sep=',')
print(bikes.head())
bikes.head()

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [8]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [11]:
df = pd.read_excel('../../data/Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [0]:
#ANSWER


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [12]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-07-05T22:05:38.936Z,44.1496,-115.0592,10.0,2.9,ml,NaN,76.00,0.274,0.78,...,2020-07-05T22:15:11.040Z,"12 km SW of Stanley, Idaho",earthquake,1.3,2.0,0.055,44.0,reviewed,us,us
1,2020-07-05T22:04:34.260Z,38.1410,-118.0478,6.1,2.5,ml,15.0,89.16,0.068,0.15,...,2020-07-05T22:10:22.131Z,"28 km S of Mina, Nevada",earthquake,NaN,0.7,NaN,NaN,automatic,nn,nn
2,2020-07-05T21:38:53.435Z,57.2641,-150.9541,4.9,3.0,ml,NaN,NaN,NaN,0.72,...,2020-07-05T21:44:22.335Z,"88 km ESE of Chiniak, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [3]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                             Bank Name           City  ST   CERT  \
 0                 The First State Bank  Barboursville  WV  14361   
 1                   Ericson State Bank        Ericson  NE  18265   
 2     City National Bank of New Jersey         Newark  NJ  21111   
 3                        Resolute Bank         Maumee  OH  58317   
 4                Louisa Community Bank         Louisa  KY  58112   
 ..                                 ...            ...  ..    ...   
 556                 Superior Bank, FSB       Hinsdale  IL  32646   
 557                Malta National Bank          Malta  OH   6629   
 558    First Alliance Bank & Trust Co.     Manchester  NH  34264   
 559  National State Bank of Metropolis     Metropolis  IL   3815   
 560                   Bank of Honolulu       Honolulu  HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                         MVB Bank, Inc.      April 3, 2020  
 1             Farmers and Merchants Bank  F

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [4]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [7]:
#ANSWER
bikes.shape


(17379, 17)

What are the column names in `bikes`?

In [10]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [11]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [12]:
#ANSWER
bikes.index


RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [16]:
#ANSWER
print(bikes['season'])
print(bikes[['season']])

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64
       season
0           1
1           1
2           1
3           1
4           1
...       ...
17374       1
17375       1
17376       1
17377       1
17378       1

[17379 rows x 1 columns]


How would we use object notation to show the first 4 rows of `atemp`?

In [25]:
#ANSWER
bikes['atemp'].head(4)

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [26]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [29]:
#ANSWER
bikes.loc(0)


What is the correct way to access the 2nd column of the DataFrame by its index?

In [30]:
#ANSWER
bikes.loc(1)

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:
Returns TRUE if elemtet is empty.

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [31]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [39]:
#ANSWER
bikes.isnull().sum().sum()


0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [37]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [42]:
#ANSWER
def divide(num,denom):
    if (denom == 0):
        return 0
    else:
        return num/denom
print(divide(10,3))
print(divide(10,0))

3.3333333333333335
0


Apply the Pandas `isna` function to the following data objects:

In [43]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [49]:
#ANSWER
print(pd.isna(x))
print(pd.isna(y))

False
True


In [47]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [53]:
#ANSWER
print(pd.isna(array))


[[False  True False]
 [False False  True]]


AttributeError: module 'pandas' has no attribute 'na_values'

How is the pandas I/O parameter `na_values` used?

? ANSWER: 
User can sepcifocy what is trreated as a NaN vaule.

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [59]:
#ANSWER
print(bikes.head())
print(bikes.season.unique())
print(bikes.temp.unique())

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  
[1 2 3 4]
[0.24 0.22 0.2  0.32 0.38 0.36 0.42 0.46 0.44 0.4  0.34 0.3  0.26 0.16
 0.14 0.18 0.12 0.

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [69]:
#ANSWER
print(bikes.dtypes)
print(bikes.instant.min(), bikes.instant.max(), bikes.instant.max()-bikes.instant.min())
print(bikes.dteday.min(), bikes.dteday.max())
print(bikes.windspeed.min(), bikes.windspeed.max())

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object
1 17379 17378
2011-01-01 2012-12-31
0.0 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [77]:
#bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
#bikes_min, bikes_max
nbikes=bikes.select_dtypes(include=np.number)

bikes_min = nbikes.min()
bikes_max = nbikes.max()
print(bikes_max,bikes_min)

instant       17379.0000
season            4.0000
yr                1.0000
mnth             12.0000
hr               23.0000
holiday           1.0000
weekday           6.0000
workingday        1.0000
weathersit        4.0000
temp              1.0000
atemp             1.0000
hum               1.0000
windspeed         0.8507
casual          367.0000
registered      886.0000
cnt             977.0000
dtype: float64 instant       1.00
season        1.00
yr            0.00
mnth          1.00
hr            0.00
holiday       0.00
weekday       0.00
workingday    0.00
weathersit    1.00
temp          0.02
atemp         0.00
hum           0.00
windspeed     0.00
casual        0.00
registered    0.00
cnt           1.00
dtype: float64


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [78]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [80]:
#ANSWER
print(bikes['atemp'].quantile(0.1))

0.2424


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [81]:
type(bikes['atemp'])

pandas.core.series.Series

In [85]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
537,538,2011-01-24,1,0,1,13,0,1,1,2,0.14,0.1364,0.43,0.2239,6,54,60
6791,6792,2011-10-15,4,0,10,13,0,6,0,1,0.60,0.6212,0.35,0.4627,171,223,394
2935,2936,2011-05-07,2,0,5,5,0,6,0,1,0.46,0.4545,0.59,0.1343,4,3,7
948,949,2011-02-12,1,0,2,3,0,6,0,1,0.12,0.1970,0.80,0.0000,3,7,10
15473,15474,2012-10-11,4,1,10,22,0,4,1,1,0.40,0.4091,0.66,0.1642,10,167,177


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [86]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [89]:
#ANSWER
print(atemp_level)

0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]


Here is a random sample of `atemp_level`:

In [92]:
atemp_level.sample(5)          

9858        (0.25, 0.5]
13695       (0.5, 0.75]
7238        (0.25, 0.5]
437      (-0.001, 0.25]
12648       (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [93]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

13449     hot
11929    warm
12155    warm
1120     mild
12591    warm
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [103]:
#ANSWER
bikes['atemp_level'] = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
bikes.sample(10)
bikes.dtypes



instant           int64
dteday           object
season            int64
yr                int64
mnth              int64
hr                int64
holiday           int64
weekday           int64
workingday        int64
weathersit        int64
temp            float64
atemp           float64
hum             float64
windspeed       float64
casual            int64
registered        int64
cnt               int64
atemp_level    category
dtype: object

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

   Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [111]:
# Creating arrays with initial values
a = np.zeros((3))
print(a.shape)
print(a)
b = np.ones((1,3))
print(b.shape)
print(b)
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
print("c",c.shape)
print(c)
d = np.arange(4)
print("d", d.shape)
print(d)
e = np.array( [[1,2,3,4], [5,6,7,8]] )
print("e", e.shape)
print(e)

(3,)
[0. 0. 0.]
(1, 3)
[[1. 1. 1.]]
c (2, 3, 4)
[[[2 4 1 4]
  [5 9 9 4]
  [3 2 3 2]]

 [[2 2 7 3]
  [2 6 5 7]
  [8 5 8 7]]]
d (4,)
[0 1 2 3]
e (2, 4)
[[1 2 3 4]
 [5 6 7 8]]


In [0]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [128]:
rock = pd.read_table('../../data/rock.csv', header = 0,sep=',')
print(rock.head())
print(rock.shape)
print(rock.dtypes)

               Song Clean ARTIST CLEAN Release Year  \
0        Caught Up in You  .38 Special         1982   
1            Fantasy Girl  .38 Special          NaN   
2         Hold On Loosely  .38 Special         1981   
3  Rockin' Into the Night  .38 Special         1980   
4       Art For Arts Sake         10cc         1975   

                                COMBINED  First?  Year?  PlayCount  F*G  
0        Caught Up in You by .38 Special       1      1         82   82  
1            Fantasy Girl by .38 Special       1      0          3    0  
2         Hold On Loosely by .38 Special       1      1         85   85  
3  Rockin' Into the Night by .38 Special       1      1         18   18  
4              Art For Arts Sake by 10cc       1      1          1    1  
(2230, 8)
Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object


## Check Column Names

Check column names and clean.

In [115]:
print(rock.dtypes)

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object


# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [129]:
print(rock['Release Year'].isnull())
print("NaN found", rock['Release Year'].isnull().sum())
rock['Release Year'] = rock['Release Year'].fillna(0)
print(rock['Release Year'].isnull())
print("NaN found", rock['Release Year'].isnull().sum())


0       False
1        True
2       False
3       False
4       False
        ...  
2225     True
2226    False
2227    False
2228    False
2229    False
Name: Release Year, Length: 2230, dtype: bool
NaN found 577
0       False
1       False
2       False
3       False
4       False
        ...  
2225    False
2226    False
2227    False
2228    False
2229    False
Name: Release Year, Length: 2230, dtype: bool
NaN found 0


# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [139]:
print(rock.dtypes)
#rock['Release Year'] = 1 * rock['Release Year']
rock[["Release Year"]] = rock[["Release Year"]].apply(pd.to_numeric, errors='coerce')
print(rock.dtypes)

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object
Song Clean       object
ARTIST CLEAN     object
Release Year    float64
COMBINED         object
First?            int64
Year?             int64
PlayCount         int64
F*G               int64
dtype: object


# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [142]:
min = rock.min()
max = rock.max()
print(min)
print(max)

Song Clean                     #9 Dream
ARTIST CLEAN                .38 Special
Release Year                          0
COMBINED        #9 Dream by John Lennon
First?                                1
Year?                                 0
PlayCount                             0
F*G                                   0
dtype: object
Song Clean                     Ziggy Stardust
ARTIST CLEAN                             a-ha
Release Year                             2014
COMBINED        Ziggy Stardust by David Bowie
First?                                      1
Year?                                       1
PlayCount                                 142
F*G                                       142
dtype: object


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [154]:
def prow(r):
    if (r['Release Year'] < 1970):
        x = "Before 1970"
    else:
        x = "After 1970"
    print(r['Song Clean'], r['ARTIST CLEAN'], x)

prow(rock.iloc[2])

Hold On Loosely .38 Special After 1970


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [155]:
def fn(df):
    return df.apply(pd.to_numeric, errors='coerce')

## Apply these functions to your dataset

In [160]:
print(prow(rock.iloc[4]))
y = fn(rock)
print(prow(y.iloc[4]))


Art For Arts Sake 10cc After 1970
None
nan nan After 1970
None


## Describe the new float-only DataFrame.

In [161]:
y.describe()


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
count,2.000000,0.0,2229.000000,0.0,2230.0,2230.000000,2230.000000,2230.000000
mean,1012.000000,NaN,1465.988784,NaN,1.0,0.741256,16.872646,15.048430
std,1367.544515,NaN,866.834789,NaN,0.0,0.438043,25.302972,25.288366
min,45.000000,NaN,0.000000,NaN,1.0,0.000000,0.000000,0.000000
25%,528.500000,NaN,0.000000,NaN,1.0,0.000000,1.000000,0.000000
50%,1012.000000,NaN,1973.000000,NaN,1.0,1.000000,4.000000,3.000000
75%,1495.500000,NaN,1981.000000,NaN,1.0,1.000000,21.000000,18.000000
max,1979.000000,NaN,2014.000000,NaN,1.0,1.000000,142.000000,142.000000




> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



